In [9]:
import pandas as pd
import numpy as np
import scipy as sp
import scipy
import scipy.sparse
# import scipy.sparse as sp
import networkx as nx
import os
import re
import sys
import random
from collections import deque
import gc
import glob
import cooler

In [2]:
def get_partitions(adj_matrix, k):
    g = nx.from_numpy_array(adj_matrix.todense()) 
    import metispy 
    cost, parts = metispy.part_graph(g, k)
    ps  =  {}
    counter = 0
    for i in parts:
        if i in ps:
            continue
        else:
            ps[i] = counter
            counter += 1
    l = []
    for i in parts:
        l.append(ps[i])
    return l
def get_orders(parts):
    parts = np.array(parts)
    ordering = np.zeros(len(parts))
    rev_ordering = np.zeros(len(parts))
    column_counter = 0
    for part_id in range(len(set(parts))):
        group_members = np.where(parts == part_id)[0]
        for column in group_members:
            ordering[column_counter] = column
            rev_ordering[column] = column_counter
            column_counter += 1
    rev_ordering = np.array(rev_ordering, dtype = np.int)
    ordering = np.array(ordering, dtype = np.int)
    return ordering, rev_ordering
def construct_q1(m, parts, alpha):
    q1 =  np.zeros(shape = m.shape)
    parts = np.array(parts)
    m = m.todense()
    for i in range(max(parts) + 1):
        nodes = list(np.where(parts == i)[0])
        start = min(nodes)
        end = max(nodes) + 1
        submat = m[start:end, start:end]
        I = np.eye(submat.shape[0])
        q1[start:end, start:end] = np.linalg.inv(I - alpha * submat)
    return q1
def construct_w2(m, parts):
    w2 =  np.zeros(shape = m.shape)
    parts = np.array(parts)
    m = m.todense()
    for i in range(max(parts) + 1):
        nodes = list(np.where(parts == i)[0])
        start = min(nodes)
        end = max(nodes) + 1
        w2[start:end, :start] = m[start:end, :start]
        w2[start:end, end:] = m[start:end, end:]
        w2[:start, start:end] = m[:start, start:end]
        w2[end:, start:end] = m[end:, start:end]
    return w2

def solve_rwr_blin(stoch_matrix, alpha = 0.4, final_try = False, setname = None, chrom = None, num_parts = None, only_q1 = False):
    num_parts = num_parts if num_parts else 100
    parts = get_partitions(stoch_matrix, num_parts)
    ordering, rev_ordering = get_orders(parts)
    stoch_matrix = stoch_matrix[:, ordering]
    q1 = construct_q1(stoch_matrix, parts, 1-alpha)
    if only_q1:
        res = q1
    else:
        w2 = construct_w2(stoch_matrix, parts)
        U, s, V = sp.linalg.svd(w2)
        s = sp.sparse.spdiags(s, 0, stoch_matrix.shape[0], stoch_matrix.shape[0], format = "csc").todense()
        lmd = np.linalg.inv(np.linalg.inv(s) - (1 - alpha) * V @ q1 @ U)
        res = (alpha)*(q1 + (1 - alpha) * q1 @ U @ lmd @ V @ q1)
    res += res.transpose()
    res = res[:, rev_ordering]
    return res




def solve_rwr_iterative(stoch_matrix, alpha = 0.05, final_try = False, setname = None, chrom = None, max_iter = None):
    max_iter = max_iter if max_iter else float("inf")
    gc.collect()
    m = stoch_matrix
    y = sp.sparse.spdiags([1] * m.shape[0], 0, m.shape[0], m.shape[0], format = "csc")
    s = y
    delta = float("inf")
    A = y
    #print(m.todense()[:4,:4])
    counter = 0
    while delta > 1e-6 and counter < max_iter:
        #print(delta, counter)
        counter += 1
        Aold = A.copy()
        A = (1-alpha) * m * Aold + (alpha) * y
        delta = (abs(A - Aold)).max()
    #print(delta)
    A += A.transpose()
    return A


def solve_rwr_inverse(stoch_matrix, alpha=0.05, final_try=False, setname=None, chrom=None):
    gc.collect()
    m = stoch_matrix * (1 - alpha)
    m = m.transpose()

    y = scipy.sparse.spdiags([1] * m.shape[0], 0, m.shape[0], m.shape[0], format="csc")
    A = y - m
    try:
        s = None
        A = A.todense()
        y = y.todense()
        s = scipy.linalg.solve(A, y)

    except Exception as e:
        if A is not None:
            del A
        if y is not None:
            del y
        if s is not None:
            del s
        if final_try:
            gc.collect()
            raise Exception("Cannot allocate enough memory for solving RWR")
        else:
            return "try_later"
    s *= alpha
    s = s + s.transpose()
    
    if y is not None:
        del y
    if A is not None:
        del A
    if m is not None:
        del m
    return s


def rwr_impute(matrix, alpha=0.5, max_iter=None):
    row_sums = matrix.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1  # 避免除以0
    transition_matrix = matrix / row_sums
    transition_matrix_sparse = sp.sparse.csr_matrix(transition_matrix)
    imputed = solve_rwr_inverse(transition_matrix_sparse, alpha, True)
    imputed += imputed.T
    return imputed

In [3]:
target = "Cell2020"
imputed_matrix_dir = f"/shareb/mliu/evaluate_impute/data/imputed_data/snap"

In [31]:

print(target)
downsample_matrix_dir = f"/shareb/mliu/evaluate_impute/data/simulation_hic/{target}/hic/matrix"
for group in range(1,5):
    print(group)
    downsample_matrix = np.load(f"{downsample_matrix_dir}/{target}_sample_matrix{group}.npy")
    downsample_matrix = downsample_matrix[0:100,:]
    imputed_matrices = np.array([rwr_impute(mat, alpha=0.05) for mat in downsample_matrix])

    #对imputed_matrices中的每一项取-np.log2
    for mat in imputed_matrices:
        np.fill_diagonal(mat, np.nan)

    print(imputed_matrices.shape)
    #保存
    save_path = f"{imputed_matrix_dir}/snap_imputed_{target}_group{group}.npy"
    print(save_path)
    np.save(save_path, imputed_matrices)

Cell2020
1


(100, 243, 243)
/shareb/mliu/evaluate_impute/data/imputed_data/snap/snap_imputed_Cell2020_group1.npy
2
(100, 243, 243)
/shareb/mliu/evaluate_impute/data/imputed_data/snap/snap_imputed_Cell2020_group2.npy
3
(100, 243, 243)
/shareb/mliu/evaluate_impute/data/imputed_data/snap/snap_imputed_Cell2020_group3.npy
4
(100, 243, 243)
/shareb/mliu/evaluate_impute/data/imputed_data/snap/snap_imputed_Cell2020_group4.npy


In [4]:
## Real data

In [10]:
chr_list = ["chr1","chr2","chr3","chr4","chr5","chr6","chr7","chr8","chr9","chr10","chr11","chr12","chr13","chr14","chr15","chr16","chr17","chr18","chr19","chr20","chr21","chr22"]
sc_matrix_dir = f"/shareb/mliu/evaluate_impute/data/real_sc_hic/Ramani/cooler"
for chr in chr_list:
    print(chr)
    imputed_matrices_list = []
    for cell in range(0,620):
        print(f"cell{cell}")
        clr = cooler.Cooler(f"{sc_matrix_dir}/cell{cell}.cool")
        matrix_chr = clr.matrix(balance=False).fetch(chr)
        matrix_chr_imputed = rwr_impute(matrix_chr, alpha=0.05)
        imputed_matrices_list.append(matrix_chr_imputed)
    imputed_matrices_array = np.array(imputed_matrices_list)
    print(imputed_matrices_array.shape)
    save_path = f"{imputed_matrix_dir}/snap_imputed_Ramani_{chr}.npy"
    np.save(save_path, imputed_matrices_array)

chr1
cell0
cell1
cell2
cell3
cell4
cell5
cell6
cell7
cell8
cell9
cell10
cell11
cell12
cell13
cell14
cell15
cell16
cell17
cell18
cell19
cell20
cell21
cell22
cell23
cell24
cell25
cell26
cell27
cell28
cell29
cell30
cell31
cell32
cell33
cell34
cell35
cell36
cell37
cell38
cell39
cell40
cell41
cell42
cell43
cell44
cell45
cell46
cell47
cell48
cell49
cell50
cell51
cell52
cell53
cell54
cell55
cell56
cell57
cell58
cell59
cell60
cell61
cell62
cell63
cell64
cell65
cell66
cell67
cell68
cell69
cell70
cell71
cell72
cell73
cell74
cell75
cell76
cell77
cell78
cell79
cell80
cell81
cell82
cell83
cell84
cell85
cell86
cell87
cell88
cell89
cell90
cell91
cell92
cell93
cell94
cell95
cell96
cell97
cell98
cell99
cell100
cell101
cell102
cell103
cell104
cell105
cell106
cell107
cell108
cell109
cell110
cell111
cell112
cell113
cell114
cell115
cell116
cell117
cell118
cell119
cell120
cell121
cell122
cell123
cell124
cell125
cell126
cell127
cell128
cell129
cell130
cell131
cell132
cell133
cell134
cell135
cell136
cell137
c